In [7]:
import re
import ast
import json
import pickle
import pandas as pd
from tqdm import tqdm
from pprint import pprint

from urllib.parse import urlparse
from url_parser import parse_url, get_url, get_base_url

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from IPython.display import display

from sklearn.preprocessing import LabelEncoder

### 1. Break large file to 9 smaller ones: 
Run this in terminal:

```split -l 500000 -a 4 dataset.json smaller_```

Rename the files to `1 - 9.json`

### 2. Convert json to csv file with selected column names
1. first convert all the individual json files to csv files
2. get all the column names in all the files
3. select which colums to use, and save those files to a csv folder

In [15]:
def convert_json_csv():
    for i in tqdm(range(1, 10)):
        data = pd.read_json(f'../data/json/{i}.json', lines = True)
        data.to_csv(f'../data/json/{i}.csv')

def get_cols_name():
    cols = set()
    for i in tqdm(range(1, 10)):
        data = pd.read_csv(f'../data/csv/{i}.csv', low_memory = False)
        cols.update(data.columns.values.tolist())
    return cols

def use_cols_csv():
    for i in tqdm(range(1, 10)):
        data = pd.read_csv(
            f'../data/json/{i}.csv', 
            low_memory = False, 
            usecols = ['browserFamily', 'channel', 'deviceType', 'iabCategories', 'os', 'refDomain', 'url']
        )
        data.to_csv(f'../data/csv/{i}.csv')

def json_to_csv(print_cols = False):
    convert_json_csv()
    if print_cols:
        cols = get_cols_name()
        pprint(cols)
    use_cols_csv()

In [16]:
json_to_csv()

100%|█████████████████████████████████████████████| 9/9 [00:35<00:00,  3.94s/it]


### 3. Clean csv file
1. seperate labels and no labels
2. save the no labels in a seperate folder
3. explode/expand the iabcategories for each url -> one line per iabcategory per url
4. add the scores (high, medium, low) for each
5. break url into base, domain, path
6. combine the channel, domain, path together in one 
7. remove stop words (open source and custom)

In [19]:
custom_stopwords = ['www', 'www.', 'com', 'les', 'org', 'tag', 'html', 'id', 'un', 'win', 
                    'en', 'me', 'php', 'asp', 'aspx', 'cc', 'net']

def get_iab_categories(iab_categories):
    iab_categories_list = []
    iab_categories = ast.literal_eval(iab_categories)
    for iab_category in iab_categories:
        for value in list(iab_category.values())[:-1]:
            iab_categories_list.append([value, iab_category['score']])
    return iab_categories_list

def get_base_url_(url):
    try:
        return get_base_url(url)
    except:
        return url

def get_domain(url):
    try:
        domain = parse_url(url)['domain']
    except:
        domain = ' '.join(urlparse(url).netloc.split('.'))
    try:
        top_domain = parse_url(url)['top_domain'] or ''
    except:
        top_domain = ''
    try:
        sub_domain = parse_url(url)['sub_domain'] or ''
    except:
        sub_domain = ''
    return domain + ' ' + sub_domain + ' ' + top_domain 
    
def get_path(url):
    try:
        path = parse_url(url)['path']
    except:
        path = urlparse(url).path
    return path

def remove_stop_words(text):
    # add lematize
    stop = []
    stop.extend(stopwords.words('english'))
    stop.extend(stopwords.words('french'))
    stop.extend(stopwords.words('spanish'))
    stop.extend(custom_stopwords)
    text = re.sub('[^A-Za-z]+', ' ', str(text)).strip()
    text = ' '.join([word for word in text.split() if word not in (stop)])
    
    return text
    
def clean_df(df, df_name):  
    lemmatizer = WordNetLemmatizer()  
    df_nolabel = df[df['iabCategories'].isnull()]
    df_nolabel.to_csv(f'../data/clean/no_label/{df_name}.csv')
    
    df = df[~df['iabCategories'].isnull()]
    df = df.fillna('')
    df['iabCategories'] = df['iabCategories'].apply(get_iab_categories)
    df = df.explode('iabCategories', ignore_index = True)
    split = pd.DataFrame(df['iabCategories'].to_list(), columns = ['iab_categories', 'confidence'])
    df = pd.concat([df, split], axis = 1)
    df = df.drop('iabCategories', axis = 1)
    df['base_url'] = df['url'].apply(get_base_url_)
    df['domain'] = df['url'].apply(get_domain) 
    df['path'] = df['url'].apply(get_path)
    df['combine'] = df['channel'] + ' ' + df['domain'] + ' ' + df['path']
    df['combine'] = df['combine'].apply(remove_stop_words)
    df['combine'] = df['combine'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

    df.to_csv(f'../data/clean/label/{df_name}.csv')
    
def clean():
    for i in tqdm(range(1, 10)):
        df = pd.read_csv(f'../data/csv/{i}.csv')
        clean_df(df, i)

In [20]:
clean()

100%|█████████████████████████████████████████| 9/9 [4:04:58<00:00, 1633.20s/it]


### 4. Get data ready for training

1. look at all the columns
2. get rid of unnessary cols
3. look at all the unique values in certain columns
4. replace them with numerical classes
5. join all csv files into one

In [17]:
def get_cols_name():
    cols = set()
    for i in tqdm(range(1, 10)):
        df = pd.read_csv(f'../data/clean/label/{i}.csv', low_memory = False)
        cols.update(df.columns.values.tolist())
    return cols

def drop_cols():
    cols_to_drop = ['Unnamed: 0', 'base_url', 'channel', 'domain', 'path', 'refDomain', 'url']
    for i in tqdm(range(1, 10)):
        df = pd.read_csv(f'../data/clean/label/{i}.csv', low_memory = False)
        df = df.rename(columns = {'Unnamed: 0.1': 'id'})
        df = df.set_index('id')
        df = df.drop(cols_to_drop, axis = 1)
        df.to_csv(f'../data/train/{i}.csv')
        
def get_unique_vals():
    unique_vals = {
        'browserFamily': set(),
        'deviceType': set(),
        'os': set(),
        'confidence': set()
    }
    for i in tqdm(range(1, 10)):
        df = pd.read_csv(f'../data/train/{i}.csv', low_memory = False)
        for col in unique_vals.keys():
            unique_vals[col].update(df[col].unique())
    return unique_vals
        
    
def display_df():
    for i in range(1, 10):
        df = pd.read_csv(f'../data/train/{i}.csv', low_memory = False)
        display(df.head())
        print()
        
def combine():
    dfs = []
    for i in tqdm(range(1, 10)):
        df = pd.read_csv(f'../data/train/{i}.csv', low_memory = False)
        dfs.append(df)
    combined_df = pd.concat(dfs, axis=0, ignore_index = True)
    combined_df.to_csv('../data/train/train_combined.csv', index = False)

def group_val():
    with open('../static_data/browser_family.pkl', 'rb') as f:
        browser_family = pickle.load(f)
    with open('../static_data/os.pkl', 'rb') as f:
        os = pickle.load(f)
    df = pd.read_csv('../data/train/train_combined.csv', low_memory = False)
    df['browserFamily'] = df['browserFamily'].apply(lambda x: browser_family[x])
    df['os'] = df['os'].apply(lambda x: os[x])
    df.to_csv('../data/train/train_grouped.csv', index = False)
    
def replace_val_le():
    df = pd.read_csv('../data/train/train_grouped.csv', low_memory = False)
    
    le = LabelEncoder()
    le.fit(df['browserFamily'])
    le_browser_family_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    with open('../data/static_data/browser_family_mapping.pkl', 'wb') as f:
        pickle.dump(le_browser_family_mapping, f)
    df['browserFamily'] = le.transform(df['browserFamily'])
    
    le = LabelEncoder()
    le.fit(df['deviceType'])
    le_device_type_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    with open('../data/static_data/device_type_mapping.pkl', 'wb') as f:
        pickle.dump(le_device_type_mapping, f)
    df['deviceType'] = le.transform(df['deviceType'])
    
    le = LabelEncoder()
    le.fit(df['os'])
    le_os_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    with open('../data/static_data/os_mapping.pkl', 'wb') as f:
        pickle.dump(le_os_mapping, f)
    df['os'] = le.transform(df['os'])
    
    le = LabelEncoder()
    le.fit(df['iab_categories'])
    le_iab_categories_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    with open('../data/static_data/iab_categories_mapping.pkl', 'wb') as f:
        pickle.dump(le_iab_categories_mapping, f)
    df['iab_categories'] = le.transform(df['iab_categories'])
    
    df['confidence'] = df['confidence'].replace({'medium': 0.5, 'high': 1.0,})

    df = df.dropna()
    
    df.to_csv('../data/train/train.csv', index = False)

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    try:
        text = str(text)
        text = ' '.join([lemmatizer.lemmatize(list(str(word))) for word in text.split()])
        return text
    except:
        return text

def replace_val():
    df = pd.read_csv('../data/train/train_grouped.csv', low_memory = False)
    
    le_browser_family_mapping = pickle.load(open('../static_data/browser_family_mapping.pkl', 'rb'))
    le_device_type_mapping = pickle.load(open('../static_data/device_type_mapping.pkl', 'rb'))
    le_os_mapping = pickle.load(open('../static_data/os_mapping.pkl', 'rb'))
    le_iab_categories_mapping = pickle.load(open('../static_data/iab_categories_mapping.pkl', 'rb'))

    df['browserFamily'] = df['browserFamily'].replace(le_browser_family_mapping)
    df['deviceType'] = df['deviceType'].replace(le_device_type_mapping)
    df['os'] = df['os'].replace(le_os_mapping)
    df['iab_categories'] = df['iab_categories'].replace(le_iab_categories_mapping)
    df['confidence'] = df['confidence'].replace({'medium': 0.5, 'high': 1.0,})
    df['combine'] = df['combine'].apply(lemmatize)

    df = df.dropna()
    
    df.to_csv('../data/train/train.csv', index = False)

def get_train(show_cols = False, show_unique_vals = False):
    # if show_cols:
    #     pprint(get_cols_name())
    # drop_cols()
    # if show_unique_vals:
    #     pprint(get_unique_vals())
    # combine()
    # group_val()
    replace_val()

In [18]:
get_train()

In [20]:
def get_train_alternate():
    lemmatizer = WordNetLemmatizer()
    cols_to_drop = ['browserFamily', 'deviceType', 'os', 'combine']
    le_iab_categories_mapping = pickle.load(open('../static_data/iab_categories_mapping.pkl', 'rb'))

    df = pd.read_csv('../data/train/train_combined.csv', low_memory = False)
    df = df.fillna('')
    df['X'] = df['browserFamily'].str.lower() + ' ' + df['deviceType'].str.lower() + ' ' + df['os'].str.lower() + ' ' + df['combine'].str.lower()
    df['X'] = df['X'].apply(lemmatize)

    df = df.drop(cols_to_drop, axis = 1)
    
    df['confidence'] = df['confidence'].replace({'medium': 0.5, 'high': 1.0,})
    df['iab_categories'] = df['iab_categories'].replace(le_iab_categories_mapping)
    df.to_csv('../data/train/train_alternate.csv', index = False)

get_train_alternate()

In [19]:
df = pd.read_csv('../data/train/train.csv')
pprint(df.shape)
df.sample(n = 10)

(9143493, 7)


,id,browserFamily,deviceType,os,iab_categories,confidence,combine
8133959,882044,3,5.0,3,13,1.0,flixhd genre romance
3138929,33187,3,6.0,3,140,0.5,churchmouseyarns collections free patterns
1579845,543285,3,5.0,3,150,1.0,tn onedhs gov csp
8599636,311265,0,3.0,5,166,1.0,NaN
390188,390188,0,3.0,7,95,0.5,sukafilmxxi online silent night
1851417,814857,3,5.0,3,157,0.5,face faceafrica article drug lord made k order...
8021581,769666,3,5.0,0,151,1.0,prambors site movie pirates caribbean worlds end
6922507,709272,3,5.0,0,142,1.0,incestflix FD page
2970309,899845,0,3.0,5,95,1.0,americanthinker
7835111,583196,3,5.0,3,60,1.0,inverse entertainment time travel shows januar...


In [22]:
df = pd.read_csv('../data/train/train_alternate.csv')
pprint(df.shape)
df.sample(n = 10)

(9143493, 4)


,id,iab_categories,confidence,X
5920754,741712,147,0.5,mobile safari smartphone ios elitedaily entert...
8324386,36015,150,1.0,chrome mobile smartphone ios bncollege mga login
8857808,569437,142,1.0,mobile safari smartphone ios face faceafrica a...
5172396,1030997,96,1.0,mobile safari smartphone ios face faceafrica a...
8980773,692402,64,1.0,chrome mobile smartphone android hotelcaesiust...
3486096,380354,95,1.0,mobile safari smartphone ios unitedwater
2269816,199352,17,1.0,safari personal computer os x localconditions ...
6691436,478201,13,1.0,mobile safari tablet ios google search naijapr...
7089890,876655,65,1.0,mobile safari smartphone ios usatoday bengalsw...
2713383,642919,155,1.0,mobile safari smartphone ios anime watch emine...
